In [ ]:
# importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from warnings import filterwarnings
filterwarnings('ignore')
%matplotlib inline

In [ ]:
# importando o conjunto de dados
df = pd.read_csv('dataset_bike/hour.csv')

In [ ]:
# verificando as primeiras linhas
df.head()

In [ ]:
# verificando as informações das features
df.info()

In [ ]:
# analisando as informações estatísticas
df.describe()

In [ ]:
# distribuição da nossa variável target
plt.figure(figsize=(12,8))
sns.set_style('darkgrid')
sns.distplot(df.cnt)

In [ ]:
# verificando a correlação

vmin_corr = -0.3
vmax_corr = 0.3

correlation = df.corr()
correlation[(correlation['cnt'] <= vmin_corr)|(correlation['cnt'] >= vmax_corr)]['cnt']

Após definir um intervalo para aparecer somente as correlações mais fortes, podemos notar que as features com correlações mais fortes (casual e registered) diz a mesma coisa que a variável target, portanto se eu mantê-las estarei tendenciando o meu modelo.

In [ ]:
# excluindo as variáveis que não fazem sentido para o meu modelo
# rodar com todas as variáveis possíveis

col_drop = ['dteday', 'casual', 'registered', 'cnt']
df_modified = df.drop(col_drop, axis=1)
df_modified.head()

In [ ]:
# constantes
SEED = 42
TEST_SIZE = 0.3

In [ ]:
# Separando em dados de treino e teste
X = df_modified
y = df.cnt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state=SEED)

In [ ]:
# instanciando os estimadores
lr = LinearRegression()
dt = DecisionTreeRegressor()

In [ ]:
# treinando o modelo
lr_model = lr.fit(X_train, y_train)

# realizando predição
y_pred = lr_model.predict(X_test)

# calculando o erro médio quadrático
print(f'O erro médio quadrado do Linear Regression: {mean_squared_error(y_test, y_pred)}')
print(f'\nO Score, ou seja o quanto que a variação do modelo explica a variável resposta: {lr_model.score(X_test, y_test)*100:.2f}%')
print('**Quanto maior o percentual do Score, mais ajustado à linha de Regressão é o modelo**')

In [ ]:
# afim de verificar também como foi a performance nos dados de treino

# realizando predição
y_pred_train = lr_model.predict(X_train)

# calculando o erro médio quadrático
print(f'O erro médio quadrado do Linear Regression: {mean_squared_error(y_train, y_pred_train)}')
print(f'\nO Score, ou seja o quanto que a variação do modelo explica a variável resposta: {lr_model.score(X_train, y_train)*100:.2f}%')
print('**Quanto maior o percentual do Score, mais ajustado à linha de Regressão é o modelo**')

In [ ]:
# instanciando os modelos
lr = LinearRegression()
dt = DecisionTreeRegressor()
rf = RandomForestRegressor()

# criando uma lista dos instanciadores
list_model = [('Linear_Regression', lr),
              ('Decision_Tree', dt),
              ('Random_Forest', rf)]

# treinando e testando os modelos
print('AVALIANDO OS MODELOS COM TRAIN TEST SPLIT:')
print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')
for name, model in list_model:
  trained = model.fit(X_train, y_train)
  y_pred = trained.predict(X_test)
  print(f'Resultado do modelo {name}')
  print('----------------------------------------------------------------------------------------')
  print(f'O erro médio quadrado do {name}: {mean_squared_error(y_test, y_pred)}')
  print(f'O Score, ou seja o quanto que a variação do modelo explica a variável resposta: {trained.score(X_test, y_test)*100:.2f}%')
  print('----------------------------------------------------------------------------------------\n')

In [ ]:
# ajustando os hiperparâmetros do Random Forest
# especificando os parâmetros

parameters = {'min_samples_leaf':[1,10],
              'min_samples_split':[2,10],
              'n_estimators':[100,500,1000]}

In [ ]:
# instanciando os parametros de tunning
rf = RandomForestRegressor()
grid = GridSearchCV(rf, parameters)

In [ ]:
# treinando com Grid
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
rf_best = grid.best_estimator_

In [ ]:
# gerando o resultado final
final_result = rf_best.predict(X_test)

In [ ]:
# calculando o resultado
print(f'O erro médio quadrado do Grid: {mean_squared_error(y_test, final_result)}')
print(f'O Score, ou seja o quanto que a variação do modelo explica a variável resposta: {rf_best.score(X_test, y_test)*100:.2f}%')